In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time


In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_12916\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 136.09 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #5s
            dataX = dataX5
            dataY = dataY5
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
        else:
            #1s
            dataX = dataX1
            dataY = dataY1
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.783967971801758 Train_acc: 3.062500%
Train Epoch: 2/60 Train_Loss: 3.4752371311187744 Train_acc: 4.812500%
Train Epoch: 3/60 Train_Loss: 3.489485263824463 Train_acc: 6.187500%
Train Epoch: 4/60 Train_Loss: 3.366658926010132 Train_acc: 5.250000%
Train Epoch: 5/60 Train_Loss: 3.524843454360962 Train_acc: 6.437500%
Train Epoch: 6/60 Train_Loss: 3.394404172897339 Train_acc: 8.312500%
Train Epoch: 7/60 Train_Loss: 3.637580156326294 Train_acc: 10.187500%
Train Epoch: 8/60 Train_Loss: 3.079935312271118 Train_acc: 8.375000%
Train Epoch: 9/60 Train_Loss: 3.246304750442505 Train_acc: 12.625000%
Train Epoch: 10/60 Train_Loss: 3.205681324005127 Train_acc: 10.437500%
Train Epoch: 11/60 Train_Loss: 2.720015287399292 

Train Epoch: 47/60 Train_Loss: 0.5046240091323853 Train_acc: 75.416672%
Train Epoch: 48/60 Train_Loss: 0.5871164798736572 Train_acc: 75.500000%
Train Epoch: 49/60 Train_Loss: 0.5310877561569214 Train_acc: 75.416672%
Train Epoch: 50/60 Train_Loss: 0.8003435134887695 Train_acc: 76.270836%
Train Epoch: 51/60 Train_Loss: 0.9087805151939392 Train_acc: 75.854172%
Train Epoch: 52/60 Train_Loss: 0.8132578730583191 Train_acc: 75.708336%
Train Epoch: 53/60 Train_Loss: 0.5481503009796143 Train_acc: 75.666672%
Train Epoch: 54/60 Train_Loss: 0.7063732743263245 Train_acc: 76.062500%
Train Epoch: 55/60 Train_Loss: 0.8993796110153198 Train_acc: 76.250000%
Train Epoch: 56/60 Train_Loss: 0.9888643622398376 Train_acc: 76.395836%
Train Epoch: 57/60 Train_Loss: 0.7607924342155457 Train_acc: 76.958336%
Train Epoch: 58/60 Train_Loss: 0.8663612008094788 Train_acc: 76.770836%
Train Epoch: 59/60 Train_Loss: 0.8206082582473755 Train_acc: 76.895836%
Train Epoch: 60/60 Train_Loss: 0.9328124523162842 Train_acc: 77.

Train Epoch: 31/60 Train_Loss: 1.6342854499816895 Train_acc: 49.250000%
Train Epoch: 32/60 Train_Loss: 1.4666653871536255 Train_acc: 48.812500%
Train Epoch: 33/60 Train_Loss: 1.8478513956069946 Train_acc: 49.437500%
Train Epoch: 34/60 Train_Loss: 1.606645941734314 Train_acc: 50.250000%
Train Epoch: 35/60 Train_Loss: 1.6157845258712769 Train_acc: 49.312500%
Train Epoch: 36/60 Train_Loss: 1.4005333185195923 Train_acc: 52.125000%
Train Epoch: 37/60 Train_Loss: 1.3368310928344727 Train_acc: 51.187500%
Train Epoch: 38/60 Train_Loss: 1.430546522140503 Train_acc: 53.187500%
Train Epoch: 39/60 Train_Loss: 1.459592342376709 Train_acc: 52.187500%
Train Epoch: 40/60 Train_Loss: 1.5848464965820312 Train_acc: 52.500000%
Train Epoch: 41/60 Train_Loss: 1.6232765913009644 Train_acc: 51.062500%
Train Epoch: 42/60 Train_Loss: 1.8633095026016235 Train_acc: 52.000000%
Train Epoch: 43/60 Train_Loss: 1.1902508735656738 Train_acc: 53.812500%
Train Epoch: 44/60 Train_Loss: 1.5425069332122803 Train_acc: 56.625

Train Epoch: 15/60 Train_Loss: 1.544031023979187 Train_acc: 53.062504%
Train Epoch: 16/60 Train_Loss: 1.4653825759887695 Train_acc: 59.312504%
Train Epoch: 17/60 Train_Loss: 1.1754088401794434 Train_acc: 61.700005%
Train Epoch: 18/60 Train_Loss: 1.2461872100830078 Train_acc: 61.875004%
Train Epoch: 19/60 Train_Loss: 1.2253656387329102 Train_acc: 62.850002%
Train Epoch: 20/60 Train_Loss: 0.9774846434593201 Train_acc: 64.000000%
Train Epoch: 21/60 Train_Loss: 1.2152742147445679 Train_acc: 62.825005%
Train Epoch: 22/60 Train_Loss: 1.3569605350494385 Train_acc: 63.975002%
Train Epoch: 23/60 Train_Loss: 0.9561579823493958 Train_acc: 63.687504%
Train Epoch: 24/60 Train_Loss: 0.9910673499107361 Train_acc: 64.625000%
Train Epoch: 25/60 Train_Loss: 1.3606016635894775 Train_acc: 63.512505%
Train Epoch: 26/60 Train_Loss: 1.3587480783462524 Train_acc: 64.675003%
Train Epoch: 27/60 Train_Loss: 1.2783831357955933 Train_acc: 63.800003%
Train Epoch: 28/60 Train_Loss: 1.0029398202896118 Train_acc: 65.2

Train Epoch: 1/60 Train_Loss: 1.688905954360962 Train_acc: 33.541668%
Train Epoch: 2/60 Train_Loss: 1.8005627393722534 Train_acc: 44.333336%
Train Epoch: 3/60 Train_Loss: 1.7773035764694214 Train_acc: 44.270836%
Train Epoch: 4/60 Train_Loss: 1.7271240949630737 Train_acc: 45.895836%
Train Epoch: 5/60 Train_Loss: 2.0798192024230957 Train_acc: 46.625000%
Train Epoch: 6/60 Train_Loss: 2.070291519165039 Train_acc: 49.187500%
Train Epoch: 7/60 Train_Loss: 1.7371972799301147 Train_acc: 49.166668%
Train Epoch: 8/60 Train_Loss: 1.5505374670028687 Train_acc: 50.937500%
Train Epoch: 9/60 Train_Loss: 1.9111396074295044 Train_acc: 50.520836%
Train Epoch: 10/60 Train_Loss: 1.4374786615371704 Train_acc: 53.333336%
Train Epoch: 11/60 Train_Loss: 0.9443314671516418 Train_acc: 52.770836%
Train Epoch: 12/60 Train_Loss: 1.637375831604004 Train_acc: 54.250000%
Train Epoch: 13/60 Train_Loss: 1.2468022108078003 Train_acc: 56.708336%
Train Epoch: 14/60 Train_Loss: 1.3318579196929932 Train_acc: 55.000000%
Trai

Train Epoch: 50/60 Train_Loss: 0.6558247804641724 Train_acc: 79.400002%
Train Epoch: 51/60 Train_Loss: 0.6750139594078064 Train_acc: 79.762505%
Train Epoch: 52/60 Train_Loss: 0.6415491104125977 Train_acc: 79.725006%
Train Epoch: 53/60 Train_Loss: 0.693943202495575 Train_acc: 79.925003%
Train Epoch: 54/60 Train_Loss: 0.8407555222511292 Train_acc: 79.862503%
Train Epoch: 55/60 Train_Loss: 1.0841517448425293 Train_acc: 80.237503%
Train Epoch: 56/60 Train_Loss: 0.46375131607055664 Train_acc: 80.325005%
Train Epoch: 57/60 Train_Loss: 0.8943503499031067 Train_acc: 80.025002%
Train Epoch: 58/60 Train_Loss: 0.5923086404800415 Train_acc: 80.300003%
Train Epoch: 59/60 Train_Loss: 0.5599971413612366 Train_acc: 80.887505%
Train Epoch: 60/60 Train_Loss: 0.6182652711868286 Train_acc: 80.562508%
Test accuracy of the model: 37.850000%
Test accuracy of the model: 42.666667%
Test accuracy of the model: 42.250000%
Training Time: 51.61 seconds
可用的 GPU 数量: 1
------------------------------------------------

Train Epoch: 34/60 Train_Loss: 0.604360044002533 Train_acc: 76.895836%
Train Epoch: 35/60 Train_Loss: 0.6032851934432983 Train_acc: 77.187500%
Train Epoch: 36/60 Train_Loss: 0.7995449304580688 Train_acc: 76.666672%
Train Epoch: 37/60 Train_Loss: 0.8215924501419067 Train_acc: 77.395836%
Train Epoch: 38/60 Train_Loss: 0.7293862104415894 Train_acc: 78.312500%
Train Epoch: 39/60 Train_Loss: 0.9266420006752014 Train_acc: 78.812500%
Train Epoch: 40/60 Train_Loss: 0.7108092308044434 Train_acc: 78.687500%
Train Epoch: 41/60 Train_Loss: 0.6930923461914062 Train_acc: 77.250000%
Train Epoch: 42/60 Train_Loss: 0.7433086037635803 Train_acc: 78.645836%
Train Epoch: 43/60 Train_Loss: 0.5354067087173462 Train_acc: 79.645836%
Train Epoch: 44/60 Train_Loss: 0.7620238065719604 Train_acc: 78.062500%
Train Epoch: 45/60 Train_Loss: 0.7074329853057861 Train_acc: 79.187500%
Train Epoch: 46/60 Train_Loss: 0.5741174817085266 Train_acc: 81.458336%
Train Epoch: 47/60 Train_Loss: 0.6306982636451721 Train_acc: 82.6

Train Epoch: 18/60 Train_Loss: 1.9855469465255737 Train_acc: 35.812500%
Train Epoch: 19/60 Train_Loss: 2.0270278453826904 Train_acc: 38.187500%
Train Epoch: 20/60 Train_Loss: 1.9082129001617432 Train_acc: 39.062500%
Train Epoch: 21/60 Train_Loss: 1.9508322477340698 Train_acc: 40.125000%
Train Epoch: 22/60 Train_Loss: 2.0547921657562256 Train_acc: 37.187500%
Train Epoch: 23/60 Train_Loss: 1.7142062187194824 Train_acc: 41.312500%
Train Epoch: 24/60 Train_Loss: 1.7953914403915405 Train_acc: 40.687500%
Train Epoch: 25/60 Train_Loss: 2.1483848094940186 Train_acc: 43.687500%
Train Epoch: 26/60 Train_Loss: 1.7584387063980103 Train_acc: 42.687500%
Train Epoch: 27/60 Train_Loss: 2.0467073917388916 Train_acc: 43.937500%
Train Epoch: 28/60 Train_Loss: 1.87088143825531 Train_acc: 45.187500%
Train Epoch: 29/60 Train_Loss: 1.964537501335144 Train_acc: 45.812500%
Train Epoch: 30/60 Train_Loss: 1.5170915126800537 Train_acc: 45.000000%
Train Epoch: 31/60 Train_Loss: 1.6030093431472778 Train_acc: 49.125

Train Epoch: 2/60 Train_Loss: 2.241118907928467 Train_acc: 44.937504%
Train Epoch: 3/60 Train_Loss: 1.6698564291000366 Train_acc: 42.750004%
Train Epoch: 4/60 Train_Loss: 1.8977940082550049 Train_acc: 48.050003%
Train Epoch: 5/60 Train_Loss: 1.6897025108337402 Train_acc: 47.212502%
Train Epoch: 6/60 Train_Loss: 2.023495674133301 Train_acc: 46.275002%
Train Epoch: 7/60 Train_Loss: 2.1074488162994385 Train_acc: 49.325001%
Train Epoch: 8/60 Train_Loss: 1.9324067831039429 Train_acc: 50.137501%
Train Epoch: 9/60 Train_Loss: 2.1821417808532715 Train_acc: 49.775002%
Train Epoch: 10/60 Train_Loss: 2.0806729793548584 Train_acc: 50.975002%
Train Epoch: 11/60 Train_Loss: 1.7160091400146484 Train_acc: 51.187504%
Train Epoch: 12/60 Train_Loss: 2.137439012527466 Train_acc: 51.462502%
Train Epoch: 13/60 Train_Loss: 2.0407423973083496 Train_acc: 48.625004%
Train Epoch: 14/60 Train_Loss: 2.07656192779541 Train_acc: 51.662502%
Train Epoch: 15/60 Train_Loss: 1.6375374794006348 Train_acc: 51.700001%
Train

In [9]:
def getMax(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return max(TL2)

def getMin(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return min(TL2)

def getMean(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return np.mean(TL2)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMax(kFoldTrainAccuracy), getMax(kFoldTestAccuracy1), getMax(kFoldTestAccuracy3), getMax(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMin(kFoldTrainAccuracy), getMin(kFoldTestAccuracy1), getMin(kFoldTestAccuracy3), getMin(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMean(kFoldTrainAccuracy), getMean(kFoldTestAccuracy1), getMean(kFoldTestAccuracy3), getMean(kFoldTestAccuracy5)))

f.close()